# MVI
## Graph Neural Networks
### Binary classification model in Cheminformatics
Jozef Fulop

# Load Libraries

In [ ]:
# Standard libraries
import os
import json
import pickle
import warnings
warnings.filterwarnings('ignore', category=UserWarning, message='TypedStorage is deprecated') # Known issue with PyTorch and DGL

# Data handling
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

# Machine learning and model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, auc, confusion_matrix, classification_report
from sklearn.cluster import HDBSCAN

# Neural Networks and Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

# Graph Neural Networks
import dgl
import dgl.nn as dglnn
from dgl import batch
from dgl.data.utils import save_graphs, load_graphs

# Cheminformatics
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Draw
from dgllife.utils import mol_to_bigraph, CanonicalAtomFeaturizer, CanonicalBondFeaturizer, mol_to_graph
from chembl_structure_pipeline import standardizer

# Network analysis
import networkx as nx

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns


check if GPU is aviable and the device

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("Is CUDA Supported?", torch.cuda.is_available())

In [ ]:
torch.cuda.is_available(), torch.cuda.device_count(), torch.cuda.get_device_name(0)

# Data Preprocessing

load dataset

In [ ]:
# load json data from data_mvi folder
chemdiv = pd.read_json('data_mvi/chemdiv_df.json')
enamine = pd.read_json('data_mvi/enamine_df.json')
enmine_protein = pd.read_json('data_mvi/enamine_protein_df.json')
life_chemicals = pd.read_json('data_mvi/life_chemicals_df.json')
robin = pd.read_json('data_mvi/robin_df.json')

add source column

In [ ]:
# add 'source column'
chemdiv['source'] = 'chemdiv'
enamine['source'] = 'enamine'
enmine_protein['source'] = 'enmine_protein'
life_chemicals['source'] = 'life_chemicals'
robin['source'] = 'robin'

In [ ]:
robin.shape

compute fingerprings for algorithms

In [ ]:
# combine all dataframes
df = pd.concat([chemdiv, enamine, enmine_protein, life_chemicals, robin], ignore_index=True)
# delte 'mol' column and ECFp6 column
df = df.drop(['mol', 'ECFP6'], axis=1)

# create 'mol' column and use SMILES column to create mol object
df['mol'] = df['SMILES'].apply(lambda x: Chem.MolFromSmiles(x))
# create 'ECFP6' column and use mol object to create ECFP6 fingerprint
df['ECFP6'] = df['mol'].apply(lambda x: AllChem.GetMorganFingerprintAsBitVect(x, 3, nBits=2048))


these fingerprints will be further used more in data preprocessing,train and test set selection. (not in this milestone). Dataset was deduplicated before.

In [ ]:
# check for duplicates using 'SMILES' column
df.duplicated(subset=['SMILES']).sum()

In [ ]:
df.head()

create column for labels, if 'rna' == 1 - molecule binds to the RNA, else - Protein

In [ ]:
# create 'rna' column and if in column 'source' is 'enmine_protein' set 'rna' to 0 else 1
df['rna'] = df['source'].apply(lambda x: 0 if x == 'enmine_protein' else 1)

In [ ]:
df.shape

dataset is unbalanced - creating balanced dataset - undersampling majority class (just for now, in near future more sofisticated approaches will be used - clustering)

In [ ]:
chemdiv.shape, enamine.shape, enmine_protein.shape, life_chemicals.shape, robin.shape

In [ ]:
# divide data back to 5 dataframes
chemdiv = df[df['source'] == 'chemdiv']
enamine = df[df['source'] == 'enamine']
enmine_protein = df[df['source'] == 'enmine_protein']
life_chemicals = df[df['source'] == 'life_chemicals']
robin = df[df['source'] == 'robin']

rna_df = pd.concat([chemdiv, enamine, life_chemicals, robin], ignore_index=True)

rna_df.shape

In [ ]:
enmine_protein.head()

In [ ]:
df.shape

sanity check 

In [ ]:
# show rows from df with each source so there will be 5 rows and all columns
df.groupby('source').head(1)


In [ ]:
enmine_protein_small = enmine_protein.sample(36937)


In [ ]:
# combine all df
df_small = pd.concat([chemdiv, enamine, enmine_protein_small, life_chemicals, robin], ignore_index=True)

In [ ]:
df_small.groupby('source').head(1)

In [ ]:
df_small['rna'].value_counts()

Check for the inconsistency


In [ ]:
# drop SMILES column
df_small = df_small.drop(['SMILES'], axis=1)

In [ ]:
df_small['mol'] = df_small['mol'].apply(lambda mol: standardizer.standardize_mol(mol))

In [ ]:
# compute SMILES for each molecule
df_small['SMILES'] = df_small['mol'].apply(lambda mol: Chem.MolToSmiles(mol))

In [ ]:
from rdkit import Chem
import pandas as pd

# Initialize a list to store molecules with disconnected components
disconnected_mols = []

# Check for valid SMILES and Mols
for idx, row in df_small.iterrows():
    smiles = row['SMILES']
    mol = row['mol']
    source = row['source']

    # Check if the SMILES string can be converted to a valid RDKit Mol object
    mol_from_smiles = Chem.MolFromSmiles(smiles)
    if mol_from_smiles is None:
        print(f"Invalid SMILES string at index {idx}: {smiles}")
        continue

    # Check if the Mol object is valid (no disconnected components)
    if mol is None or mol.GetNumAtoms() == 0:
        print(f"Invalid Mol object at index {idx}: {smiles}")
        continue

    # Check for disconnected components in Mol objects
    if mol is not None:
        # Get the number of disconnected components
        num_components = Chem.GetMolFrags(mol, asMols=False, sanitizeFrags=False)
        if len(num_components) > 1:
            print(f"Disconnected components in molecule at index {idx}: {smiles}, from source {source}")
            disconnected_mols.append(row)
            df_small.drop(idx, inplace=True)

# Convert the list to a DataFrame
disconnected_mols_df = pd.DataFrame(disconnected_mols)


In [ ]:
df_small.shape

## Create Graphs from molecules and add features to nodes (atoms) - mainly one hot encoding , edges (bonds)

In [ ]:
def create_graphs_from_dataframe(df):
    graphs = []
    labels = []

    for index, row in df.iterrows():
        mol = row['mol']  # RDKit Mol object
        label = row['rna']  # Binary label

        # Convert RDKit molecule to a bi-directed DGLGraph and featurize
        graph = mol_to_bigraph(mol,  # mol_to_bigraph
                               node_featurizer=CanonicalAtomFeaturizer(),
                               edge_featurizer=CanonicalBondFeaturizer(self_loop=False),
                               explicit_hydrogens=False,
                               add_self_loop=False)
               
        # graph = dgl.add_self_loop(graph)
        graphs.append(graph)
        labels.append(label)

    return graphs, torch.tensor(labels).unsqueeze(1)


# Create graphs and labels from the DataFrame
graphs, labels = create_graphs_from_dataframe(df_small)

# Now, 'graphs' is a list of DGLGraphs, and 'labels' is a tensor of labels 
# corresponding to each graph. We can use DGL's batch utility function to
# batch these graphs together into a single large graph. This is a common
# technique for training GNNs on multiple graphs.

save graphs - tensors

In [ ]:
graph_labels = df_small[['source', 'rna']].to_dict('records')  # Convert relevant columns to a list of dictionaries

# Save the graphs
dgl.save_graphs('data_mvi/molecule_graphs_orig.bin', graphs)

# Save the labels and additional information
with open('graph_labels.json', 'w') as f:
    json.dump(graph_labels, f)


In [ ]:
# Load the graphs
loaded_graphs_orig, _ = dgl.load_graphs('data_mvi/molecule_graphs_orig.bin')

# Load the labels and additional information
with open('graph_labels.json', 'r') as f:
    loaded_graph_labels = json.load(f)

# You can now reconstruct a DataFrame or directly use the loaded data
reconstructed_df = pd.DataFrame(loaded_graph_labels)


### Visualization of the graph representation 

In [ ]:
def visualize_dgl_graph(dgl_graph):
    g = dgl.to_networkx(dgl_graph)  # Convert DGL graph to a NetworkX graph
    pos = nx.spring_layout(g)  # Position nodes using Fruchterman-Reingold force-directed algorithm
    nx.draw(g, pos, with_labels=False, node_color='skyblue', node_size=200, edge_color='black')
    plt.show()

# Visualize the first graph in the list
visualize_dgl_graph(loaded_graphs_orig[42])

In [ ]:

mol = df_small['mol'][42]
AllChem.Compute2DCoords(mol)
# draw the molecule
Draw.MolToImage(mol, size=(600, 600))

# Create train and test set

In [ ]:
# First split into training and test sets
train_graphs, test_graphs, train_labels, test_labels = train_test_split(
    loaded_graphs_orig, 
    reconstructed_df['rna'], 
    test_size=0.2, 
    random_state=42
)

# Further split the training set into training and validation sets
train_graphs, val_graphs, train_labels, val_labels = train_test_split(
    train_graphs, 
    train_labels, 
    test_size=0.2,  # 20% of the original training set for validation
    random_state=42
)

# merge train and val,  labels are Series
train_graphs_all = train_graphs + val_graphs
train_labels_all = pd.concat([train_labels, val_labels])

there was a problem with few isolated nodes in few graphs so I added loops to each node to have information at least about themselfs

In [ ]:
for i, g in enumerate(train_graphs):
    if (g.in_degrees() == 0).any():
        print(f"Graph {i} has isolated nodes")

# if there is not any isolated nodes in train_graphs, print 'No isolated nodes'
print('No isolated nodes')

for i, g in enumerate(test_graphs):
    if (g.in_degrees() == 0).any():
        print(f"Graph {i} has isolated nodes")

# if there is not any isolated nodes in train_graphs, print 'No isolated nodes'
print('No isolated nodes')

# Graph Convolutional Network Model (GCN)

In [ ]:
class GCNClassifier(nn.Module):
    def __init__(self, in_feats, hidden_size):
        super(GCNClassifier, self).__init__()
        self.conv1 = dglnn.GraphConv(in_feats, hidden_size)
        self.conv2 = dglnn.GraphConv(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, g, features):
        x = F.relu(self.conv1(g, features))
        x = F.relu(self.conv2(g, x))
        g.ndata['h'] = x
        x = dgl.mean_nodes(g, 'h')
        return self.fc(x)


## Instance of the the GCN Model

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model
in_feats = train_graphs[0].ndata['h'].shape[1]  # Replace with your feature size
model = GCNClassifier(in_feats, hidden_size=148).to(device)  # hidden_size is set to 74,  num_classes=2

# Loss and optimizer
# criterion = nn.CrossEntropyLoss() 
criterion = nn.BCEWithLogitsLoss()  # Use this loss if you are using a sigmoid activation
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)



## Train the GCN model

In [ ]:
class GraphDataset(Dataset):
    def __init__(self, graphs, labels):
        self.graphs = graphs
        
        # Reset index if labels is a pandas DataFrame/Series to ensure continuous indexing
        if isinstance(labels, pd.DataFrame) or isinstance(labels, pd.Series):
            self.labels = labels.reset_index(drop=True)
        else:
            self.labels = labels

    def __len__(self):
        # The lengths of graphs and labels should be the same
        assert len(self.graphs) == len(self.labels), "Graphs and labels must have the same length"
        return len(self.graphs)

    def __getitem__(self, idx):
        try:
            graph = self.graphs[idx]
            label = self.labels[idx]
            return graph, label
        except IndexError:
            print(f"IndexError: Index {idx} out of range for dataset of size {len(self)}")
            raise
        
        

In [ ]:
def collate_graphs(samples):
    graphs, labels = map(list, zip(*samples))
    batched_graph = dgl.batch(graphs)
    labels = torch.tensor(labels, dtype=torch.float32)
    return batched_graph, labels



In [ ]:
batch_size = 1024
num_workers = 10

# Assuming train_labels and test_labels are originally 1D tensors or lists
# Create the datasets with updated labels
# Create the DataLoader with multiple workers

if isinstance(train_labels, torch.Tensor):
    train_labels = train_labels.tolist()
train_dataset = GraphDataset(train_graphs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers, collate_fn=collate_graphs)

if isinstance(test_labels, torch.Tensor):
    test_labels = test_labels.tolist()
test_dataset = GraphDataset(test_graphs, test_labels)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_graphs)

if isinstance(val_labels, torch.Tensor):
    val_labels = val_labels.tolist()
val_dataset = GraphDataset(val_graphs, val_labels)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False, num_workers=num_workers, collate_fn=collate_graphs)

if isinstance(train_labels_all, torch.Tensor):
    train_labels_all = train_labels_all.tolist()
train_dataset_all = GraphDataset(train_graphs_all, train_labels_all)
train_loader_all = DataLoader(train_dataset_all, batch_size=batch_size, shuffle=False, num_workers=num_workers, collate_fn=collate_graphs)



In [ ]:
def train(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_graphs, batch_labels in data_loader:
        batch_graphs = batch_graphs.to(device)
        
        # batch_labels = torch.tensor(batch_labels, dtype=torch.long).to(device)
        batch_labels = batch_labels.clone().detach().to(device)
        batch_features = batch_graphs.ndata['h']

        outputs = model(batch_graphs, batch_features)
        outputs = outputs.squeeze() # remove extra dimension
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(data_loader)



In [ ]:
def validate(model, data_loader, criterion, device):
    model.eval()
    total_loss = 0
    y_true = []
    y_pred = []
    y_pred_proba = []
    with torch.no_grad():
        for graphs, labels in data_loader:
            graphs = graphs.to(device)
            features = graphs.ndata['h'].to(device)
            labels = labels.to(device)
            outputs = model(graphs, features)
            if outputs.dim() == 2 and outputs.shape[0] == 1:
                outputs = outputs.squeeze(0)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            # Check the dimension of outputs before applying torch.max
            if outputs.dim() > 1:
                _, predicted = torch.max(outputs.data, 1)
            else:
                predicted = outputs.round()  # Assuming binary classification

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_pred_proba.extend(outputs.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    return avg_loss, y_true, y_pred, y_pred_proba



In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.epochs_no_improve = 0
        self.early_stop = False
        self.stopped_epoch = 0  # Attribute to store the epoch number

    def __call__(self, val_loss, epoch):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.epochs_no_improve = 0
        else:
            self.epochs_no_improve += 1
            if self.epochs_no_improve >= self.patience:
                self.early_stop = True
                self.stopped_epoch = epoch  # Store the epoch number



In [ ]:
num_epochs = 500
early_stopping = EarlyStopping(patience=20, min_delta=0.002)

# Initialize lists to store loss values
gcn_train_loss_list = []
gcn_val_loss_list = []

for epoch in range(num_epochs):
    # Training
    train_loss = train(model, train_loader, criterion, optimizer, device)
    gcn_train_loss_list.append(train_loss)  # Store train loss

    # Validation
    val_loss, _, _, _ = validate(model, val_loader, criterion, device)
    gcn_val_loss_list.append(val_loss)  # Store validation loss

    # Print and check for early stopping
    print(f"Epoch: {epoch+1}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")
    early_stopping(val_loss, epoch)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break


# Store the epoch number when early stopping was triggered
num_epochs_early = early_stopping.stopped_epoch + 1



In [ ]:
# Ensure that the length of gcn_train_loss_list and gcn_val_loss_list is equal to the number of epochs processed
assert len(gcn_train_loss_list) == len(gcn_val_loss_list)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(gcn_train_loss_list, label='Training Loss')
plt.plot(gcn_val_loss_list, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Train on merged train and val sets


In [ ]:
def train(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_graphs, batch_labels in data_loader:
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.clone().detach().to(device)
        batch_features = batch_graphs.ndata['h']

        outputs = model(batch_graphs, batch_features)
        outputs = outputs.squeeze() # remove extra dimension
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(data_loader)


## Reset weights before training again with all data


In [ ]:
def reset_weights(m):
    """
    This function will reset the weights of a given module.
    """
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

# Example usage with your model
model.apply(reset_weights)


In [ ]:
print("Final training on merged training and validation sets")

# Initialize list to store loss values
final_train_losses = []

for epoch in range(num_epochs_early):
    # Training
    train_loss = train(model, train_loader_all, criterion, optimizer, device)
    final_train_losses.append(train_loss)  # Store train loss

    # Print train loss
    print(f"Epoch {epoch+1}/{num_epochs_early}, Train Loss: {train_loss:.4f}")


In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(final_train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs on Merged Training and Validation Sets')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


## Save the model

In [ ]:
# if there is not a folder named 'model' create one
if not os.path.exists('model_experiment'):
    os.mkdir('model_experiment')

# save the model
torch.save(model.state_dict(), 'model_experiment/gcn_model_experiment.pth')

In [ ]:
# load the model
model = GCNClassifier(in_feats, hidden_size=148).to(device)
model.load_state_dict(torch.load('model_experiment/gcn_model_experiment.pth'))

## Test the model

In [ ]:
def test(model, data_loader, device):
    model.eval()
    y_true = []
    y_pred = []
    y_pred_proba = []

    with torch.no_grad():
        for graphs, labels in data_loader:
            graphs = graphs.to(device)
            features = graphs.ndata['h'].to(device)
            labels = labels.to(device)

            outputs = model(graphs, features)
            probabilities = torch.sigmoid(outputs).squeeze()  # Apply sigmoid to convert logits to probabilities

            predicted = (probabilities > 0.5).long()  # Convert probabilities to binary predictions

            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())
            y_pred_proba.extend(probabilities.cpu().numpy())

    return y_true, y_pred, y_pred_proba


## Predict probabilities

In [ ]:
# Now you can use this DataLoader in your test function
y_true, y_pred, y_pred_proba = test(model, test_loader, device)

# Convert probabilities to percentages for the positive class
# Assuming binary classification and class '1' is the positive class
confidence_percentages = [prob * 100 for prob in y_pred_proba]

# Now 'confidence_percentages' holds the confidence level for each prediction


In [ ]:
# Extract the second element (index 1) from each 1D array in y_pred_proba
# y_pred_proba = [prob[1] for prob in y_pred_proba]
# Now y_pred_proba is a list of probabilities for the positive class
# Convert it to a NumPy array
y_pred_proba = np.array(y_pred_proba)


## Results

In [ ]:
print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
print(f'Precision: {precision_score(y_true, y_pred)}')
print(f'Recall: {recall_score(y_true, y_pred)}')
print(f'F1: {f1_score(y_true, y_pred)}')
print(f'AUC: {roc_auc_score(y_true, y_pred_proba)}')


In [ ]:
# Accuracy: 0.7896446700507614
# Precision: 0.8110537951363301
# Recall: 0.7508527766407422
# F1: 0.7797931132209155
# AUC: 0.8738676601333576

## Confusion matrix

In [ ]:
# create folder 'visuals' if it does not exist
if not os.path.exists('visuals'):
    os.mkdir('visuals')
    

In [ ]:
# confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=['RNA binder', 'non-RNA binder'], columns=['RNA binder', 'non-RNA binder'])
sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
sns.set(context='paper', style='white', font_scale=1.5)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('visuals/confusion_matrix_gcn_experiment.png', dpi=300)
plt.show()



## Classification report

In [ ]:
# classification report
print(classification_report(y_true, y_pred))


## ROC AUC

In [ ]:
# visualize ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)'% roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.savefig('visuals/roc_curve_gcn_experiment.png', dpi=300)
plt.show()


In [ ]:
# visualize confidence_percentages in plot
plt.figure(figsize=(8, 6))
plt.hist(confidence_percentages, bins=50)
plt.title('Confidence Histogram')
plt.xlabel('Confidence Percentage')
plt.ylabel('Frequency')
plt.savefig('visuals/confidence_histogram_gcn_experiment.png', dpi=300)
plt.show()


In [ ]:
# Sort test predictions by confidence and get top 10 indices
sorted_pairs = sorted(enumerate(confidence_percentages), key=lambda x: x[1], reverse=True)
top_molecules_test_indices = [index for index, _ in sorted_pairs[:20]]

# The indices in test_graphs correspond to their positions in the original loaded_graphs_orig
# Find the original indices in loaded_graphs_orig
original_indices = [loaded_graphs_orig.index(test_graphs[i]) for i in top_molecules_test_indices]

# Use the original indices to look up the corresponding rows in reconstructed_df
top_molecule_row_indices = reconstructed_df.index[original_indices]

# Now, top_molecule_row_indices contains the DataFrame indices of the top 10 RNA-binding molecules


In [ ]:
top_molecule_row_indices

In [ ]:
# create folder 'top_molecules' if it does not exist
if not os.path.exists('top_molecules_gcn_experiment'):
    os.mkdir('top_molecules_gcn_experiment')
    

In [ ]:
# Retrieve the SMILES codes and source information for the top 10 molecules
top_smiles = df_small.iloc[top_molecule_row_indices]['SMILES'].tolist()
top_sources = df_small.iloc[top_molecule_row_indices]['source'].tolist()

# Retrieve the confidence scores using the indices relative to the test dataset
top_confidences = [confidence_percentages[i] for i in top_molecules_test_indices]

# Convert SMILES to RDKit Mol objects
mols = [Chem.MolFromSmiles(smile) for smile in top_smiles]

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(500, 500))
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    # title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    # plt.title(title)
    plt.subplots_adjust(bottom=0.20)  # Adjust the position of the title
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_gcn_experiment/molecule_{i+1}_{top_sources[i]}_a.png", dpi=300)
    plt.show()


In [ ]:
# Define the figure size and font size for titles
fig_size = (6, 6)
title_font_size = 12

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(300, 300))
    
    # Create a figure with adjusted size
    plt.figure(figsize=fig_size)
    plt.imshow(img)
    
    # Adjust the position of the text within the image
    title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    plt.title(title, fontsize=title_font_size)
    plt.subplots_adjust(bottom=0.15)  # Adjust the position of the title
    
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_gcn_experiment/molecule_{i+1}_{top_sources[i]}_b.png", dpi=300)
    # plt.show()


# Message Passing Neural Network (MPNN)

In [ ]:
class MPNNClassifier(nn.Module):
    def __init__(self, in_feats, hidden_size, out_feats):
        super(MPNNClassifier, self).__init__()
        self.edge_conv = dglnn.EdgeConv(in_feats, hidden_size)
        self.reduce_function = dgl.function.mean('m', 'h')
        self.fc = nn.Linear(74, out_feats)

    def forward(self, g, features):
        # Custom message function
        def message_function(edges):
            return {'m': edges.src['h'] + edges.dst['h']}

        # Message passing
        g.ndata['h'] = features
        g.apply_edges(message_function)
        g.update_all(dgl.function.copy_e('m', 'm'), self.reduce_function)
        
        x = dgl.mean_nodes(g, 'h')
        return self.fc(x)


## Instance of the the MPNN Model

In [ ]:
# Instantiate the model
in_feats = train_graphs[0].ndata['h'].shape[1]  # Replace with your feature size
model = MPNNClassifier(in_feats, hidden_size=74, out_feats=1).to(device)

# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Use this loss for binary classification with a sigmoid activation
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)


## Train the MPNN model

In [ ]:
num_epochs = 500
early_stopping = EarlyStopping(patience=25, min_delta=0.002)

# Initialize lists to store loss values
mpnn_train_loss_list = []
mpnn_val_loss_list = []

for epoch in range(num_epochs):
    # Training
    train_loss = train(model, train_loader, criterion, optimizer, device)
    mpnn_train_loss_list.append(train_loss)  # Store train loss

    # Validation
    val_loss, _, _, _ = validate(model, val_loader, criterion, device)
    mpnn_val_loss_list.append(val_loss)  # Store validation loss

    # Print and check for early stopping
    print(f"Epoch: {epoch+1}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")
    early_stopping(val_loss, epoch)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break
    
# Store the epoch number when early stopping was triggered
num_epochs_early = early_stopping.stopped_epoch + 1



In [ ]:
# Ensure that the length of mpnn_train_loss_list and mpnn_val_loss_list is equal to the number of epochs processed
assert len(mpnn_train_loss_list) == len(mpnn_val_loss_list)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(mpnn_train_loss_list, label='Training Loss')
plt.plot(mpnn_val_loss_list, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Train on merged train and val sets


In [ ]:
def train(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_graphs, batch_labels in data_loader:
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.clone().detach().to(device)
        batch_features = batch_graphs.ndata['h']

        outputs = model(batch_graphs, batch_features)
        outputs = outputs.squeeze() # remove extra dimension
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(data_loader)

## Reset weights before training again with all data


In [ ]:
def reset_weights(m):
    """
    This function will reset the weights of a given module.
    """
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

# Example usage with your model
model.apply(reset_weights)

In [ ]:
print("Final training on merged training and validation sets")

# Initialize list to store loss values
final_train_losses = []

for epoch in range(num_epochs_early):
    # Training
    train_loss = train(model, train_loader_all, criterion, optimizer, device)
    final_train_losses.append(train_loss)  # Store train loss

    # Print train loss
    print(f"Epoch {epoch+1}/{num_epochs_early}, Train Loss: {train_loss:.4f}")


In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(final_train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs on Merged Training and Validation Sets')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


## Save the model

In [ ]:
# if there is not a folder named 'model' create one
if not os.path.exists('model_experiment'):
    os.mkdir('model_experiment')

# save the model
torch.save(model.state_dict(), 'model_experiment/mpnn_model_experiment.pth')

In [ ]:
# Define the number of output features for binary classification
out_feats = 1

# Instantiate the model
hidden_size = 74 
model = MPNNClassifier(in_feats, hidden_size, out_feats).to(device)

# Load the model state
model.load_state_dict(torch.load('model_experiment/mpnn_model_experiment.pth'))


## Test the model

## Predict probabilities

In [ ]:
y_true, y_pred, y_pred_proba = test(model, test_loader, device)

confidence_percentages = [prob * 100 for prob in y_pred_proba]

y_pred_proba = np.array(y_pred_proba)

## Results

In [ ]:
print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
print(f'Precision: {precision_score(y_true, y_pred)}')
print(f'Recall: {recall_score(y_true, y_pred)}')
print(f'F1: {f1_score(y_true, y_pred)}')
print(f'AUC: {roc_auc_score(y_true, y_pred_proba)}')

In [ ]:
# Accuracy: 0.6711336717428088
# Precision: 0.6515337423312884
# Recall: 0.7245190339746214
# F1: 0.6860908327411331
# AUC: 0.7276068962734443

## Confusion matrix

In [ ]:
# confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=['RNA binder', 'non-RNA binder'], columns=['RNA binder', 'non-RNA binder'])
sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
sns.set(context='paper', style='white', font_scale=1.5)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('visuals/confusion_matrix_mpnn_experiment.png', dpi=300)
plt.show()



## Classification report

In [ ]:
# classification report
print(classification_report(y_true, y_pred))


## ROC AUC

In [ ]:
# visualize ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)'% roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.savefig('visuals/roc_curve_mpnn_experiment.png', dpi=300)
plt.show()


In [ ]:
# visualize confidence_percentages in plot
plt.figure(figsize=(8, 6))
plt.hist(confidence_percentages, bins=50)
plt.title('Confidence Histogram')
plt.xlabel('Confidence Percentage')
plt.ylabel('Frequency')
plt.savefig('visuals/confidence_histogram_mpnn_experiment.png', dpi=300)
plt.show()


In [ ]:
# Sort test predictions by confidence and get top 10 indices
sorted_pairs = sorted(enumerate(confidence_percentages), key=lambda x: x[1], reverse=True)
top_molecules_test_indices = [index for index, _ in sorted_pairs[:20]]

# The indices in test_graphs correspond to their positions in the original loaded_graphs_orig
# Find the original indices in loaded_graphs_orig
original_indices = [loaded_graphs_orig.index(test_graphs[i]) for i in top_molecules_test_indices]

# Use the original indices to look up the corresponding rows in reconstructed_df
top_molecule_row_indices = reconstructed_df.index[original_indices]

# Now, top_molecule_row_indices contains the DataFrame indices of the top 10 RNA-binding molecules


In [ ]:
# create folder 'top_molecules' if it does not exist
if not os.path.exists('top_molecules_mpnn_experiment'):
    os.mkdir('top_molecules_mpnn_experiment')
    

In [ ]:
# Retrieve the SMILES codes and source information for the top 10 molecules
top_smiles = df_small.iloc[top_molecule_row_indices]['SMILES'].tolist()
top_sources = df_small.iloc[top_molecule_row_indices]['source'].tolist()

# Retrieve the confidence scores using the indices relative to the test dataset
top_confidences = [confidence_percentages[i] for i in top_molecules_test_indices]

# Convert SMILES to RDKit Mol objects
mols = [Chem.MolFromSmiles(smile) for smile in top_smiles]

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(500, 500))
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    # title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    # plt.title(title)
    plt.subplots_adjust(bottom=0.20)  # Adjust the position of the title
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_mpnn_experiment/molecule_{i+1}_{top_sources[i]}_a.png", dpi=300)
    plt.show()


In [ ]:
# Define the figure size and font size for titles
fig_size = (6, 6)
title_font_size = 12

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(300, 300))
    
    # Create a figure with adjusted size
    plt.figure(figsize=fig_size)
    plt.imshow(img)
    
    # Adjust the position of the text within the image
    title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    plt.title(title, fontsize=title_font_size)
    plt.subplots_adjust(bottom=0.15)  # Adjust the position of the title
    
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_mpnn_experiment/molecule_{i+1}_{top_sources[i]}_b.png", dpi=300)
    # plt.show()


# Graph Attention Networks (GAT)

In [ ]:
class GATClassifier(nn.Module):
    def __init__(self, in_feats, hidden_size, num_heads=1):
        super(GATClassifier, self).__init__()
        self.conv1 = dglnn.GATConv(in_feats, hidden_size, num_heads=num_heads)

        # Adjusted to account for the mean aggregation of head outputs
        self.fc = nn.Linear(hidden_size, 1)  # For binary classification

    def forward(self, g, features):
        # Apply GAT convolution
        gat_output = self.conv1(g, features)  # Shape: [num_nodes, num_heads, hidden_size]

        # Mean over heads
        x = F.elu(gat_output.mean(1))  # Now shape: [num_nodes, hidden_size]
        
        # Update node features
        g.ndata['h'] = x

        # Aggregate node features across the graph
        x = dgl.mean_nodes(g, 'h')

        # Pass through the fully connected layer
        return self.fc(x)


## Instance of the the GAT Model

In [ ]:
# Instantiate the model
in_feats = train_graphs[0].ndata['h'].shape[1]  # Replace with your feature size
model = GATClassifier(in_feats, hidden_size=148, num_heads=4).to(device)  # hidden_size is set to 74

# Loss and optimizer
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()  # Use this loss if you are using a sigmoid activation
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

## Train the GAT model

In [ ]:
num_epochs = 500
early_stopping = EarlyStopping(patience=25, min_delta=0.002)

# Initialize lists to store loss values
gat_train_loss_list = []
gat_val_loss_list = []

for epoch in range(num_epochs):
    # Training
    train_loss = train(model, train_loader, criterion, optimizer, device)
    gat_train_loss_list.append(train_loss)  # Store train loss

    # Validation
    val_loss, _, _, _ = validate(model, val_loader, criterion, device)
    gat_val_loss_list.append(val_loss)  # Store validation loss

    # Print and check for early stopping
    print(f"Epoch: {epoch+1}, train_loss: {train_loss:.4f}, val_loss: {val_loss:.4f}")
    early_stopping(val_loss, epoch)
    if early_stopping.early_stop:
        print(f"Early stopping triggered at epoch {epoch+1}")
        break
    
# Store the epoch number when early stopping was triggered
num_epochs_early = early_stopping.stopped_epoch + 1



In [ ]:
# Ensure that the length of gat_train_loss_list and gat_val_loss_list is equal to the number of epochs processed
assert len(gat_train_loss_list) == len(gat_val_loss_list)

# Plotting
plt.figure(figsize=(10, 6))
plt.plot(gat_train_loss_list, label='Training Loss')
plt.plot(gat_val_loss_list, label='Validation Loss')
plt.title('Training and Validation Loss Over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## Train on merged train and val sets


In [ ]:
def train(model, data_loader, criterion, optimizer, device):
    model.train()
    total_loss = 0
    for batch_graphs, batch_labels in data_loader:
        batch_graphs = batch_graphs.to(device)
        batch_labels = batch_labels.clone().detach().to(device)
        batch_features = batch_graphs.ndata['h']

        outputs = model(batch_graphs, batch_features)
        outputs = outputs.squeeze() # remove extra dimension
        loss = criterion(outputs, batch_labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    return total_loss / len(data_loader)

## Reset weights before training again with all data


In [ ]:
def reset_weights(m):
    """
    This function will reset the weights of a given module.
    """
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
        m.reset_parameters()

# Example usage with your model
model.apply(reset_weights)

In [ ]:
print("Final training on merged training and validation sets")

# Initialize list to store loss values
final_train_losses = []

for epoch in range(num_epochs_early):
    # Training
    train_loss = train(model, train_loader_all, criterion, optimizer, device)
    final_train_losses.append(train_loss)  # Store train loss

    # Print train loss
    print(f"Epoch {epoch+1}/{num_epochs_early}, Train Loss: {train_loss:.4f}")


In [ ]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(final_train_losses, label='Training Loss')
plt.title('Training Loss Over Epochs on Merged Training and Validation Sets')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


## Save the model

In [ ]:
# if there is not a folder named 'model' create one
if not os.path.exists('model_experiment'):
    os.mkdir('model_experiment')

# save the model
torch.save(model.state_dict(), 'model_experiment/gat_model_experiment.pth')

In [ ]:
# load the model
model = GATClassifier(in_feats, hidden_size=148, num_heads=4).to(device)
model.load_state_dict(torch.load('model_experiment/gat_model_experiment.pth'))

## Test the model

## Predict probabilities

In [ ]:
y_true, y_pred, y_pred_proba = test(model, test_loader, device)

confidence_percentages = [prob * 100 for prob in y_pred_proba]

y_pred_proba = np.array(y_pred_proba)

## Results

In [ ]:
print(f'Accuracy: {accuracy_score(y_true, y_pred)}')
print(f'Precision: {precision_score(y_true, y_pred)}')
print(f'Recall: {recall_score(y_true, y_pred)}')
print(f'F1: {f1_score(y_true, y_pred)}')
print(f'AUC: {roc_auc_score(y_true, y_pred_proba)}')

In [ ]:
# Accuracy: 0.7387478849407784
# Precision: 0.7532486494378742
# Recall: 0.7039159503342884
# F1: 0.727747213993511
# AUC: 0.8162738424254579


## Confusion matrix

In [ ]:
# confusion matrix
cm = confusion_matrix(y_true, y_pred)
cm_df = pd.DataFrame(cm, index=['RNA binder', 'non-RNA binder'], columns=['RNA binder', 'non-RNA binder'])
sns.heatmap(cm_df, annot=True, cmap='Blues', fmt='g')
sns.set(context='paper', style='white', font_scale=1.5)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.savefig('visuals/confusion_matrix_gat_experiment.png', dpi=300)
plt.show()



## Classification report

In [ ]:
# classification report
print(classification_report(y_true, y_pred))


## ROC AUC

In [ ]:
# visualize ROC curve
fpr, tpr, thresholds = roc_curve(y_true, y_pred_proba)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)'% roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.title('ROC curve')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.savefig('visuals/roc_curve_gat_experiment.png', dpi=300)
plt.show()


In [ ]:
# visualize confidence_percentages in plot
plt.figure(figsize=(8, 6))
plt.hist(confidence_percentages, bins=50)
plt.title('Confidence Histogram')
plt.xlabel('Confidence Percentage')
plt.ylabel('Frequency')
plt.savefig('visuals/confidence_histogram_gat_experiment.png', dpi=300)
plt.show()


In [ ]:
# Sort test predictions by confidence and get top 10 indices
sorted_pairs = sorted(enumerate(confidence_percentages), key=lambda x: x[1], reverse=True)
top_molecules_test_indices = [index for index, _ in sorted_pairs[:20]]

# The indices in test_graphs correspond to their positions in the original loaded_graphs_orig
# Find the original indices in loaded_graphs_orig
original_indices = [loaded_graphs_orig.index(test_graphs[i]) for i in top_molecules_test_indices]

# Use the original indices to look up the corresponding rows in reconstructed_df
top_molecule_row_indices = reconstructed_df.index[original_indices]

# Now, top_molecule_row_indices contains the DataFrame indices of the top 10 RNA-binding molecules


In [ ]:
# create folder 'top_molecules' if it does not exist
if not os.path.exists('top_molecules_gat_experiment'):
    os.mkdir('top_molecules_gat_experiment')
    

In [ ]:
# Retrieve the SMILES codes and source information for the top 10 molecules
top_smiles = df_small.iloc[top_molecule_row_indices]['SMILES'].tolist()
top_sources = df_small.iloc[top_molecule_row_indices]['source'].tolist()

# Retrieve the confidence scores using the indices relative to the test dataset
top_confidences = [confidence_percentages[i] for i in top_molecules_test_indices]

# Convert SMILES to RDKit Mol objects
mols = [Chem.MolFromSmiles(smile) for smile in top_smiles]

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(500, 500))
    plt.figure(figsize=(5, 5))
    plt.imshow(img)
    # title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    # plt.title(title)
    plt.subplots_adjust(bottom=0.20)  # Adjust the position of the title
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_gat_experiment/molecule_{i+1}_{top_sources[i]}_a.png", dpi=300)
    plt.show()


In [ ]:
# Define the figure size and font size for titles
fig_size = (6, 6)
title_font_size = 12

# Generate and save images for each molecule
for i, mol in enumerate(mols):
    img = Draw.MolToImage(mol, size=(300, 300))
    
    # Create a figure with adjusted size
    plt.figure(figsize=fig_size)
    plt.imshow(img)
    
    # Adjust the position of the text within the image
    title = f"SMILES: {top_smiles[i]}\nSource: {top_sources[i]}\nConfidence: {top_confidences[i]:.2f}%"
    plt.title(title, fontsize=title_font_size)
    plt.subplots_adjust(bottom=0.15)  # Adjust the position of the title
    
    plt.axis('off')
    
    # Save the image to the 'top_molecules' folder using the rank and source as the filename
    plt.savefig(f"top_molecules_gat_experiment/molecule_{i+1}_{top_sources[i]}_b.png", dpi=300)
    plt.show()


In [ ]:
# Plot the training and validation loss, same model will have same color but different line style use seaborn and style report and second plot style to talk-presentation
sns.set(context='talk', rc={'figure.figsize': (11.5, 7)})

plt.plot(gcn_train_loss_list, label='GCN Train', color='r', linestyle='--')
plt.plot(gcn_val_loss_list, label='GCN Validation', color='r', linestyle='-')
plt.plot(mpnn_train_loss_list, label='MPNN Train', color='b', linestyle='--')
plt.plot(mpnn_val_loss_list, label='MPNN Validation', color='b', linestyle='-')
plt.plot(gat_train_loss_list, label='GAT Train', color='g', linestyle='--')
plt.plot(gat_val_loss_list, label='GAT Validation', color='g', linestyle='-')

# Markers for early stopping
plt.scatter(49, mpnn_train_loss_list[48], color='b', marker='o', s=20, label='MPNN Early Stopping')
plt.scatter(49, mpnn_val_loss_list[48], color='b', marker='o', s=20)
plt.scatter(49, gat_train_loss_list[48], color='g', marker='o', s=20, label='GAT Early Stopping')
plt.scatter(49, gat_val_loss_list[48], color='g', marker='o', s=20)
plt.scatter(80, gcn_train_loss_list[79], color='r', marker='o', s=20, label='GCN Early Stopping')
plt.scatter(80, gcn_val_loss_list[79], color='r', marker='o', s=20)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('visuals/loss_talk_experiment.png', dpi=300)
plt.show()


# Plot the training and validation loss, same model will have same color but different line style use seaborn and style report and second plot style to talk-presentation
sns.set(context='paper', rc={'figure.figsize': (10, 4)})


plt.plot(gcn_train_loss_list, label='GCN Train', color='r', linestyle='--')
plt.plot(gcn_val_loss_list, label='GCN Validation', color='r', linestyle='-')
plt.plot(mpnn_train_loss_list, label='MPNN Train', color='b', linestyle='--')
plt.plot(mpnn_val_loss_list, label='MPNN Validation', color='b', linestyle='-')
plt.plot(gat_train_loss_list, label='GAT Train', color='g', linestyle='--')
plt.plot(gat_val_loss_list, label='GAT Validation', color='g', linestyle='-')

# Markers for early stopping
plt.scatter(49, mpnn_train_loss_list[48], color='b', marker='o', s=20, label='MPNN Early Stopping')
plt.scatter(49, mpnn_val_loss_list[48], color='b', marker='o', s=20)
plt.scatter(49, gat_train_loss_list[48], color='g', marker='o', s=20, label='GAT Early Stopping')
plt.scatter(49, gat_val_loss_list[48], color='g', marker='o', s=20)
plt.scatter(80, gcn_train_loss_list[79], color='r', marker='o', s=20, label='GCN Early Stopping')
plt.scatter(80, gcn_val_loss_list[79], color='r', marker='o', s=20)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.savefig('visuals/loss_paper_experiment.png', dpi=300)
plt.show()


